In [1]:
import sys
import hddm, IPython
print('Notebook is running:', sys.executable)
from platform import python_version # further check your python version
print('The current HDDM version is', python_version())
print('The current HDDM version is', hddm.__version__) # 0.8.0
print('The current IPython version is', IPython.__version__) 

Notebook is running: /home/hcp4715/miniconda3/envs/py_ddm/bin/python
The current HDDM version is 3.7.7
The current HDDM version is 0.8.0
The current IPython version is 7.13.0


/home/hcp4715/miniconda3/envs/py_ddm/lib/python3.7/site-packages/IPython/parallel.py:13: ShimWarning: The `IPython.parallel` package has been deprecated since IPython 4.0. You should import from ipyparallel instead.
  "You should import from ipyparallel instead.", ShimWarning)


In [6]:
%matplotlib inline

# Preparation
import os, hddm, time, csv
import kabuki
from kabuki.analyze import gelman_rubin

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import date
import random

In [3]:
# define a function to run model in parallel for experiments share the same design
def run_model_5(id):
    import hddm
    
    exp_name = '5'
    print('running models %i'%id, 'for for exp', exp_name)
    dbname = 'df' + exp_name + '_chain_vtaz_%i.db'%id   # define the database name, which uses pickle format
    mname  = 'df' + exp_name + '_chain__vtaz_%i'%id     # define the name for the model
    fname  = 'df' + exp_name + '.v.hddm_stim.csv'
    df = hddm.load_csv(fname)
    m = hddm.HDDMStimCoding(df,
                            include='z', 
                            stim_col='stim', 
                            depends_on={'v':['match','val','domain'], 't':['match','val','domain'],'a':['match','val','domain']},
                            split_param='v', 
                            drift_criterion=False,
                            p_outlier=.05)
    m.find_starting_values()
    m.sample(10000, burn=5000, thin=5, dbname=dbname, db='pickle')
    m.save(mname) # save the model
    return m

In [4]:
start_time = time.time()   # the start time of the processing#
print('\nrun model for exp', '5')
from ipyparallel import Client
rc = Client()
jobs = rc[8:12].map(run_model_5, range(4)) # 4 is the number of chains
df_models = jobs.get()
print("Running 4 chains for exp", '5', "used: %f seconds." % (time.time() - start_time))



run model for exp 5
Running 4 chains for exp 5 used: 69315.499958 seconds.


NameError: name 'gelman_rubin' is not defined

In [8]:
gelman_rubin(df_models)

{'a(Emotion.Match.Bad)': 1.0000969463420162,
 'a(Emotion.Match.Good)': 0.9998244931225581,
 'a(Emotion.Match.Neutral)': 0.9997724808155434,
 'a(Emotion.Mismatch.Bad)': 1.0009233737757284,
 'a(Emotion.Mismatch.Good)': 1.0003805897450215,
 'a(Emotion.Mismatch.Neutral)': 1.0006320095168881,
 'a(Morality.Match.Bad)': 1.0025131839246029,
 'a(Morality.Match.Good)': 1.0030282504559467,
 'a(Morality.Match.Neutral)': 1.0021697062814399,
 'a(Morality.Mismatch.Bad)': 1.0013105817985926,
 'a(Morality.Mismatch.Good)': 1.0207162430977537,
 'a(Morality.Mismatch.Neutral)': 1.0003128300008644,
 'a(Person.Match.Bad)': 1.0028858393617022,
 'a(Person.Match.Good)': 1.001377419586099,
 'a(Person.Match.Neutral)': 1.002981266045473,
 'a(Person.Mismatch.Bad)': 1.0088910025112954,
 'a(Person.Mismatch.Good)': 1.0002436247268627,
 'a(Person.Mismatch.Neutral)': 1.0005200353925667,
 'a(Scene.Match.Bad)': 1.0019947108023894,
 'a(Scene.Match.Good)': 1.0038879978705233,
 'a(Scene.Match.Neutral)': 1.0008623592254673,
 